In [1]:
import numpy as np
import gvar as gv
import sys
sys.path.append('../')
from h5glance import H5Glance 
import os
import lsqfit
import io
import yaml
import warnings  
warnings.simplefilter(action="default")
warnings.filterwarnings('ignore') 

In [2]:
os.chdir('../../nucleon_sigma/xpt')

In [3]:
import fit_analysis as fa 
import fit_routine as fit
import i_o
import priors

In [4]:
from importlib import reload
reload(i_o)
reload(fa)
reload(fit)

<module 'fit_routine' from '/home/gmoney/lqcd/nucleon_sigma/xpt/fit_routine.py'>

In [5]:
from priors import prior
prior = prior
#print(prior)

In [6]:
f = open('models.yaml', 'r')
models = yaml.load(f,Loader=yaml.FullLoader)
    #print(keys)
model_info = {}
for item in models.values():
        model_info =    item['m_p:s_n2lo:d_n4lo:x_n2lo:l_n4lo:lc_t']
        model_info_sigma = item['proton:s_n2lo:d_n4lo:x_n2lo:l_n4lo:lc_f:sigma_t']
        #model_info_div_lam = item['proton:s_n2lo:d_n4lo:x_n2lo:l_n4lo:lc_t']
#print(model_info)

In [7]:
input_output = i_o.InputOutput()
data,ensembles = input_output.get_data(units='Fpi')
prior = input_output.make_prior(data,prior)
phys_point_data = input_output.get_data_phys_point()

In [8]:
data['eps_pi']

array([[0.649422(26), 0.650960(26), 0.650395(26), ..., 0.649048(26),
        0.651969(26), 0.650183(26)],
       [0.2263520(89), 0.2262893(88), 0.2260017(88), ..., 0.2261939(88),
        0.2276410(89), 0.2268811(89)],
       [0.3377581(98), 0.3387678(99), 0.3381898(98), ..., 0.3393848(99),
        0.3381612(98), 0.3377158(98)],
       ...,
       [0.2570492(70), 0.2569939(70), 0.2576270(70), ..., 0.2571496(70),
        0.2573032(70), 0.2567869(70)],
       [0.2834828(74), 0.2837166(74), 0.2830371(73), ..., 0.2833216(74),
        0.2832872(74), 0.2831732(74)],
       [0.3043467(75), 0.3048190(76), 0.3029248(75), ..., 0.3046443(75),
        0.3042100(75), 0.3039726(75)]], dtype=object)

In [9]:

fit_div_lam = fit.fit_routine(prior=prior, data=data, model_info=model_info)
fit_div_lam_out = fit_div_lam.fit
print(fit_div_lam_out)


ValueError: operands could not be broadcast together with shapes (17,) (17,5001) (17,) 

In [9]:
# fit_sigma = fit.fit_routine(prior=prior, data=data, model_info=model_info_sigma)
# fit_sigma_out = fit_sigma.fit
# print(fit_sigma_out.y)

In [10]:
#  fit_test = fit.fit_routine(prior=prior, data=data, model_info=model_info)
#  fit_out = fit_test.fit
#  print(fit_out.format(maxline=True))

In [11]:
from importlib import reload
reload(fa)
fit_analysis = fa.fit_analysis(phys_point_data=phys_point_data, data=data,model_info=model_info, prior=prior)
fit_analysis_sigma = fa.fit_analysis(phys_point_data=phys_point_data, data=data,model_info=model_info_sigma, prior=prior)
extrap_mass = fit_analysis.extrapolated_mass
extrap_mass_deriv = fit_analysis_sigma.extrapolated_mass
m_proton = extrap_mass['proton']
m_proton_deriv =  extrap_mass_deriv['proton']

KeyError: 'eps_pi'

In [13]:
reload(io)
m_pi = phys_point_data['m_pi']
lam_chi = phys_point_data['lam_chi']
e_pi = m_pi / lam_chi
sigma = m_pi**2 *gv.deriv(m_proton, m_pi**2)
l3 = gv.gvar('3.53(26)')
l4 = gv.gvar('4.73(10)')
sigma_analytic = e_pi/2 * (1 + e_pi**2*(5/2 - l3/2 - 2*l4 )) *m_proton_deriv
print('sigma w/ gv.deriv:',sigma)
print('sigma analytic:' ,sigma_analytic)


sigma w/ gv.deriv: 700.8(3.1)
sigma analytic: 45.1(1.5)


In [14]:
print(fit_analysis.error_budget)

KeyError: 'm_{proton,0}'